In [1]:
import requests
from bs4 import BeautifulSoup  
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
api_keys = pd.read_csv('api_keys.csv')
#api_keys.csv and .ipynb_checkpoints should to in the gitignore

In [3]:
#pd.DataFrame({'endpoint': 'yelp', 'key' : api_key}, index = [0]).to_csv('api_keys.csv', index = False)


In [4]:
yelp_api = api_keys.loc[api_keys['endpoint'] == 'yelp', 'key'].values[0]

In [5]:
business_endpoint = 'https://api.yelp.com/v3/businesses/search'

In [6]:
header_category = {'Authorization': 'bearer %s' % yelp_api}

In [60]:
params_offset = 50

In [61]:
params_category_music = {'location': 'Nashville', 'limit': 50, 'categories': 'musicvenues', 'offset': params_offset}


In [62]:
response_business_music = requests.get(url = business_endpoint, params = params_category_music, headers = header_category)


In [63]:
for b in response_business_music.json()['businesses'] :
    print(b['name'])

The Bluebird Cafe
The Local
BB King's Blues Club
Analog at Hutton Hotel
Commodore Grille
12th & Porter
Riverfront Park
Big Shotz
Cannery Ballroom
Jane’s Hideaway
Fat Kat Slim's
The End
Ms. Kelli's Karaoke Bar
Music City Bar and Grill
The East Room
Live Oak
Lucky Bastard Saloon
Dirty Little Secret
Brooklyn Bowl - Nashville
Nashville Municipal Auditorium
The Music of Nashville
YSB - UYE - Karaoke Like A Star!
V'lounge
Backstage Nashville
Brad's Shoe Museum
The Cobra
Foley 403 Records
True
Martha Rivers Ingram Center For The Performing Arts - Vanderbilt
Events at 624
Boulevard Records
Curb Cafe
Elevation Nightclub
Center Stage
The Cowan
Vanderbilt Univ Langford Auditorium
Nashville Center Stage
The Platform
Time Travelers Cafe
South By Sea


In [64]:
def extract_music_info(music):
    categories =  [a['title'] for a in music['categories']]
    ratings = music['rating']
    name = music['name']
    address = music['location']['display_address']
    phone_number = music['phone']
    latitude = music['coordinates']['latitude']
    longitude = music['coordinates']['longitude']
    try:
        price = music['price']
    except:
        price = 'no price'
    zip_code = music['location']['zip_code']
    is_closed = music['is_closed']
    output_dict_music = { 
                   'categories': categories,   
                   'ratings': ratings,
                   'name': name,
                   'display_address': address,
                   'phone': phone_number,
                   'latitude': latitude, 
                   'longitude': longitude, 
                   'price': price, 
                   'zip_code': zip_code,
                   'is_closed': is_closed}

    return output_dict_music

In [65]:
music_list = []
for increment in range(0, 200, 50) : 
    print(increment)
    params_category_music = {'location': 'Nashville', 'limit': 50, 'categories': 'musicvenues', 'offset': increment}

    response_business_music = requests.get(url = business_endpoint, params = params_category_music, headers = header_category)
    print(len(response_business_music.json()['businesses']))
    for music_data in response_business_music.json()['businesses']:
        music_dict = extract_music_info(music_data)
        music_list.append(music_dict)
        params_offset += 50

0
50
50
40
100
0
150
0


In [66]:
music_venues_nash.shape

(8, 10)

In [67]:
music_venues_nash = pd.DataFrame(music_list)
music_venues_nash#['name'].nunique()

,categories,ratings,name,display_address,phone,latitude,longitude,price,zip_code,is_closed
0,"[Music Venues, Dive Bars, American (Traditional)]",4.5,Robert's Western World,"[416 Broadway, Ste B, Nashville, TN 37203]",+16152449552,36.161029,-86.778007,$,37203,False
1,"[Music Venues, Historical Tours]",4.5,Ryman Auditorium,"[116 5th Ave N, Nashville, TN 37219]",+16158893060,36.161224,-86.778739,$$,37219,False
2,"[Cinema, Performing Arts, Music Venues]",4.5,Belcourt Theatre,"[2102 Belcourt Ave, Nashville, TN 37212]",+16158463150,36.136915,-86.801515,$,37212,False
3,"[Music Venues, Lounges]",3.5,Tootsie's Orchid Lounge,"[422 Broadway, Nashville, TN 37203]",+16157260463,36.160900,-86.778280,$$,37203,False
4,[Music Venues],4.5,The Station Inn,"[402 12th Ave S, Nashville, TN 37203]",+16152553307,36.152580,-86.784440,$,37203,False
...,...,...,...,...,...,...,...,...,...,...
85,[Music Venues],3.0,Vanderbilt Univ Langford Auditorium,"[1 Garland Ave, Nashville, TN 37240]",,36.143219,-86.804222,$,37240,False
86,[Music Venues],1.0,Nashville Center Stage,"[514 8th Ave S, Nashville, TN 37203]",+16152188625,36.153250,-86.778990,no price,37203,False
87,"[Music Venues, Juice Bars & Smoothies]",1.0,The Platform,"[1500 2nd Ave S, Nashville, TN 37210]",+16154289224,36.139003,-86.759543,no price,37210,False
88,[Music Venues],1.0,Time Travelers Cafe,"[212 Louise Ave, Nashville, TN 37203]",+16153214040,36.152020,-86.804970,no price,37203,False


In [32]:
music_venues_nash.to_csv(r'C:\Users\mia\Desktop\nss\projects\capstone\data\updated_nash_venues.csv', index = False)

## Begin Scraping for Bars

In [8]:
params_offset = 50
params_category_bars = {'location': 'Nashville', 'limit': 50, 'categories': 'bars', 'offset': params_offset}
response_business_bars = requests.get(url = business_endpoint, params = params_category_bars, headers = header_category)


In [9]:
for b in response_business_bars.json()['businesses'] :
    print(b['name'])

L.A. Jackson
Eastland Cafe
Cafe Roze
M.L.Rose Craft Beer & Burgers - Melrose
Superica
Corsair Distillery
Earnest Bar & Hideaway
Layla's Honky Tonk
Homegrown Taproom & Marketplace
The Pub Nashville
Chuy's
Legends Corner
Bavarian Bierhaus
Honky Tonk Central
Rudy's Jazz Room
Jason Aldean's Kitchen + Rooftop Bar
The Treehouse
Farm Burger Nashville
Cafe Intermezzo
Santa's Pub
Old Glory
Edley's Bar-B-Que - Sylvan Park
Trattoria Il Mulino
Cabana
12 South Taproom & Grill
Fleming's Prime Steakhouse & Wine Bar
Rippy's Bar & Grill
Henley
Fleet Street Pub
The Sutler
Bar Sovereign
Deacon's New South
Ole Red
Hops & Crafts Taproom
Suzy Wong's House of Yum
Ruth's Chris Steak House
The George Jones
Mangia Nashville
Attaboy
Del Frisco's Grille
Cerveza Jack's
Whiskey Bent Saloon
Villalba Italian Restaurant
Bootleggers Inn
Headquarters Beercade
City Winery Nashville
Nicky's Coal Fired
Chuy's
MAFIAoZA'S Nashville
Margaritaville - Nashville


In [10]:
def extract_bar_info(bar):
    categories =  [a['title'] for a in bar['categories']]
    ratings = bar['rating']
    name = bar['name']
    address = bar['location']['display_address']
    phone_number = bar['phone']
    latitude = bar['coordinates']['latitude']
    longitude = bar['coordinates']['longitude']
    try:
        price = bar['price']
    except:
        price = 'no price'
    zip_code = bar['location']['zip_code']
    is_closed = bar['is_closed']
    output_dict_bar = { 
                   'categories': categories,   
                   'ratings': ratings,
                   'name': name,
                   'display_address': address,
                   'phone': phone_number,
                   'latitude': latitude, 
                   'longitude': longitude, 
                   'price': price, 
                   'zip_code': zip_code,
                   'is_closed': is_closed}

    return output_dict_bar

In [ ]:
music_list = []
for increment in range(0, 200, 50) : 
    print(increment)
    params_category_music = {'location': 'Nashville', 'limit': 50, 'categories': 'musicvenues', 'offset': increment}

    response_business_music = requests.get(url = business_endpoint, params = params_category_music, headers = header_category)
    print(len(response_business_music.json()['businesses']))
    for music_data in response_business_music.json()['businesses']:
        music_dict = extract_music_info(music_data)
        music_list.append(music_dict)
        params_offset += 50